  # **AKADEMI EDUCATION**
# **Première cohorte (2025): Science des données et intelligence artificielle**
#### *Phase 5: PROJET DE SCIENCE DES DONNÉES*

**Noms des étudiants du groupe: Riché FLEURINORD et Micka LOUIS**   
**Rythme d’apprentissage: Autonome**  
**Date de soutenance: 27 octobre 2025**  
**Noms des instructeurs: Wedter JEROME et Geovany Batista Polo LAGUERRE**  
**Lien de l’article de blog (lien du dépôt GitHub): https://github.com/richefleuriord/Bank_Customer_Churn_Prediction.git**

# *3-Préparation des données*

## Introduction et Objectifs de la Préparation des Données

La **préparation des données** constitue une phase fondamentale dans tout projet de *Data Science*, car la qualité du modèle dépend directement de la qualité du jeu de données.  
Elle vise à transformer les données brutes issues du système bancaire en un format propre, cohérent et exploitable pour la modélisation prédictive.

Dans ce projet, l’objectif est de **préparer le jeu de données des clients bancaires** afin de construire plusieurs modèles capables de prédire le **risque de départ (churn)** des clients, en optimisant la performance et la robustesse de chaque algorithme.

---

### Objectifs spécifiques

- Nettoyer et vérifier la cohérence du jeu de données.  
- Créer de nouvelles variables pertinentes à partir des données existantes (*feature engineering*).  
- Transformer les variables catégorielles en variables numériques exploitables par les algorithmes de *Machine Learning*.  
- Mettre en place un prétraitement **adapté à chaque famille de modèles** :  
  - Normalisation et équilibrage (SMOTE) pour les modèles sensibles aux échelles et au déséquilibre (*ex. Régression Logistique*).  
  - Paramètres internes de pondération des classes pour les modèles robustes (*ex. Random Forest, XGBoost*).  
- Séparer les données en trois ensembles :  
  - **Train (70 %)** : pour l’apprentissage du modèle.  
  - **Validation (15 %)** : pour l’ajustement des hyperparamètres.  
  - **Test (15 %)** : pour l’évaluation finale.  
- Sauvegarder les objets essentiels (*scaler, colonnes finales, modèles entraînés*) pour leur réutilisation dans l’application Streamlit.

---

### Étapes principales du pipeline de préparation

1. **Vérifications de base** : détection des valeurs manquantes, doublons et inspection de la distribution de la variable cible.  
2. **Feature Engineering** : création de nouvelles variables (ratios, regroupement d’âges, interaction entre indicateurs d’activité, etc.) et suppression des variables inutiles.  
3. **Encodage des variables catégorielles** avec le *One-Hot Encoding*.  
4. **Séparation du jeu de données** en ensembles d’entraînement, de validation et de test tout en conservant la distribution de la cible.  
5. **Normalisation sélective** : utilisation du `StandardScaler` uniquement pour les modèles linéaires (Régression Logistique).  
6. **Gestion du déséquilibre** :  
   - Application de **SMOTE** dans le pipeline de la Régression Logistique.  
   - Utilisation de `class_weight='balanced'` pour le **Random Forest**.  
   - Ajustement de `scale_pos_weight` pour le **XGBoost**.  
7. **Sauvegarde des objets et jeux de données préparés** pour la phase de modélisation et le déploiement Streamlit.

---

*Cette approche garantit une préparation des données rigoureuse, flexible et adaptée à chaque modèle, assurant des prédictions plus fiables et une performance optimale du système de détection du churn.*



In [1]:
# ============================================================
# PRÉPARATION DES DONNÉES
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import joblib
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# CHARGEMENT DU JEU DE DONNÉES
# ============================================================
df = pd.read_csv("../Data/Customer-Churn-Records.csv")

print("Chargement réussi")
print("Nombre de lignes :", df.shape[0])
print("Nombre de colonnes :", df.shape[1])

# ============================================================
# VÉRIFICATIONS DE BASE
# ============================================================
print("\n--- Vérifications de base ---")
print("Valeurs manquantes totales :", df.isnull().sum().sum())
print("Doublons :", df.duplicated().sum())
print("\nRépartition de la variable cible (Exited) :")
print(df['Exited'].value_counts(normalize=True) * 100)

# ============================================================
# FEATURE ENGINEERING COMMUN
# ============================================================

print("\n--- Feature Engineering ---")

# Suppression des colonnes non pertinentes
df = df.drop(columns=["CustomerId", "RowNumber", "Surname"], errors='ignore')

# Suppression de variables inutiles ou fuyantes
df = df.drop(columns=["Complain"], errors='ignore')

# Ratio Balance / Salaire
df['BalanceToSalaryRatio'] = df['Balance'] / (df['EstimatedSalary'] + 1)

# Regroupement des âges : Jeune, Moyen, Âgé
df['AgeCategory'] = pd.cut(
    df['Age'],
    bins=[17, 35, 55, 100],
    labels=['Jeune', 'Moyen', 'Âgé']
)
df = df.drop(columns=['Age'], errors='ignore')

# Regroupement du nombre de produits
df['ProductCategory'] = df['NumOfProducts'].apply(
    lambda x: str(x) if x in [1, 2] else '3+'
)
df = df.drop(columns=['NumOfProducts'], errors='ignore')

# Autres variables dérivées
df['ActiveCredit'] = df['IsActiveMember'] * df['HasCrCard']
df['LowSatisfaction'] = (df['Satisfaction Score'] <= 2).astype(int)

# ============================================================
# ENCODAGE DES VARIABLES CATÉGORIELLES
# ============================================================
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# ============================================================
# SÉPARATION FEATURES / CIBLE
# ============================================================
X = df.drop('Exited', axis=1)
y = df['Exited']

# ============================================================
# SPLIT TRAIN / VAL / TEST
# ============================================================
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Alignement des colonnes
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Sauvegarde des colonnes finales
columns_final = list(X_train.columns)
joblib.dump(columns_final, r"C:\Users\HP\course\phase_5\Models\columns_final.pkl")

print("\n Étape de préparation commune terminée.")

# ============================================================
# PIPELINES SPÉCIFIQUES AUX MODÈLES
# ============================================================

print("\n--- Construction des pipelines par modèle ---")

# ============================================================
# PIPELINE - RÉGRESSION LOGISTIQUE (scaling + SMOTE)
# ============================================================

num_features = [
    'CreditScore', 'Tenure', 'Balance',
    'EstimatedSalary', 'BalanceToSalaryRatio',
    'Satisfaction Score', 'Point Earned'
]
num_features = [col for col in num_features if col in X_train.columns]

scaler = StandardScaler()
joblib.dump(scaler, r"C:\Users\HP\course\phase_5\Models\scaler.pkl")

pipeline_logreg = ImbPipeline(steps=[
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('model', LogisticRegression(max_iter=1000, solver='lbfgs'))
])

pipeline_logreg.fit(X_train, y_train)
joblib.dump(pipeline_logreg, r"C:\Users\HP\course\phase_5\Models\logistic_regression_pipeline.pkl")

print(" Pipeline Régression Logistique entraîné et sauvegardé.")

# ============================================================
# MODÈLE - RANDOM FOREST (pas de scaling, pas de SMOTE)
# ============================================================

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    class_weight='balanced'
)
rf.fit(X_train, y_train)
joblib.dump(rf, r"C:\Users\HP\course\phase_5\Models\random_forest_model.pkl")

print(" Modèle Random Forest entraîné et sauvegardé.")

# ============================================================
# MODÈLE - XGBOOST (pas de scaling, pas de SMOTE)
# ============================================================

neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

xgb = XGBClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6,
    random_state=42,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    n_jobs=-1
)
xgb.fit(X_train, y_train)
joblib.dump(xgb, r"C:\Users\HP\course\phase_5\Models\xgboost_model.pkl")

print("✔ Modèle XGBoost entraîné et sauvegardé.")

# ============================================================
# SAUVEGARDE DES DONNÉES POUR VALIDATION ET TEST
# ============================================================
X_train.to_csv("../Data/X_train_prepared.csv", index=False)
y_train.to_csv("../Data/y_train_prepared.csv", index=False)
X_val.to_csv("../Data/X_val_prepared.csv", index=False)
y_val.to_csv("../Data/y_val_prepared.csv", index=False)
X_test.to_csv("../Data/X_test_prepared.csv", index=False)
y_test.to_csv("../Data/y_test_prepared.csv", index=False)

print("\n Données sauvegardées avec succès.")
print(" Pipeline complet terminé avec succès.")


Chargement réussi
Nombre de lignes : 10000
Nombre de colonnes : 18

--- Vérifications de base ---
Valeurs manquantes totales : 0
Doublons : 0

Répartition de la variable cible (Exited) :
Exited
0    79.62
1    20.38
Name: proportion, dtype: float64

--- Feature Engineering ---

 Étape de préparation commune terminée.

--- Construction des pipelines par modèle ---
 Pipeline Régression Logistique entraîné et sauvegardé.
 Modèle Random Forest entraîné et sauvegardé.
✔ Modèle XGBoost entraîné et sauvegardé.

 Données sauvegardées avec succès.
 Pipeline complet terminé avec succès.
